#Four-lane

In [3]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches


#Parameter
N = [20,40,60,80,100,120,140,160,180] #amount of vehicles (density)
pt = 1 #unit time (step)
s = 100 #number of simulations
t = 500 #time that the each simulation longs
vemax = 1 #speed limit
vemin= 0 #speed limit
tM= 50 #capacity for each lane


class Vehiculo:
  id_iter = itertools.count() #to generate an id for each agent
  def __init__(self, tipo, posx, velocidad,posy, eje, estado,ejeinicial,sentido):
      self.id = next(Vehiculo.id_iter) #id
      self.t = tipo
      self.x = posx #X coordinate for location
      self.y = posy #Y coordinate
      self.vm = velocidad
      self.d = 0 #traveled distance
      self.a = 0.01 #aceleration default value
      self.m = random.choice(["vemax", "vm"]) #tag for following distance behavior, change the options considering the experiment you want to simulate
      self.f = vemax
      self.j = eje
      self.e = estado #"libre" to move or "parar" to don't
      self.jo=ejeinicial
      self.s=sentido
      self.fr="No" #tag to identify the ones who stops at green light
      self.im="No" #tag to identify the ones who ignores traffic ligths
      self.g="No" #to avoid double turn in the corner

  def mover(self,acel):
      self.a = acel
      self.vm = max(vemin, min(vemax, self.vm + self.a*pt))
      self.d =self.vm*pt


class Carril:
    def __init__(self,pos,eje,ncarril,sentido):
        self.p = pos
        self.j = eje
        self.n = ncarril
        self.s=sentido

class Semaf:
    def __init__(self,posx,posy,eje,semaforo,tipo):
        self.x = posx
        self.y = posy
        self.j = eje
        self.s = semaforo
        self.t=tipo

def iniciar(n): #to create agents of the model
  global vehiculos, filenames, carriles,semafs,ti,te,refx,refxn,refy,refyn
  filenames = []
  semafs=[]
  carriles = []
  ti=0
  te=0
  refx=0
  refxn=0
  refy=0
  refyn=0

  #to create traffic lights
  s1 = Semaf(29.5,20.75,"ejex","off","carro")
  semafs.append(s1)
  s2 = Semaf(30.7,19.8,"ejey","off","carro")
  semafs.append(s2)

  #to create lanes
  l1 = Carril(20,"ejex",1,"+")
  carriles.append(l1)
  l5 = Carril(20.5,"ejex",2,"-")
  carriles.append(l5)
  l2 = Carril(30,"ejey",1,"-")
  carriles.append(l2)
  l6 = Carril(30.5,"ejey",2,"+")
  carriles.append(l6)

  #to create vehicles
  vehiculos = []

  while len(vehiculos) < n*0.25:
      v = Vehiculo("A", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20,"ejex","libre","ejex","+")
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex" and z.s=="+"]:
          vehiculos.append(v)

  while len(vehiculos) < n*0.5:
      v = Vehiculo("A", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20.5,"ejex","libre","ejex","-")
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex" and z.s=="-"]:
          vehiculos.append(v)

  while len(vehiculos) < n*0.75:
    v = Vehiculo("A",30, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey","-")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey" and z.s=="-"]:
        vehiculos.append(v)

  while len(vehiculos) < n:
    v = Vehiculo("A",30.5, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey","+")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey" and z.s=="+"]:
        vehiculos.append(v)

  for v in vehiculos: #vehicles that don't respect traffic lights
    while len([v for v in vehiculos if v.im == "Si" and v.t != "doble"]) <0.5*n: #change the number next to the "n" to change the percent of vehicles that don't respect traffic lights
      s = random.choice([v for v in vehiculos if v.t != "doble"])
      s.im = "Si"


def parametros(v,vm):
  distancias = {
      ("vemax"): vemax,
      ("vm"): vm}
  return distancias.get((v),"NA")

def toroide(): #toroidal space
  for v in [v for v in vehiculos if v.j == "ejex" and v.s=="+"]:
    if v.x > tM:
        v.x = v.x % tM

  for v in [v for v in vehiculos if v.j == "ejex" and v.s=="-"]:
    if v.x < 0:
        v.x = v.x % tM

  for v in [v for v in vehiculos if v.j == "ejey" and v.s=="+"]:
    if v.y > tM:
        v.y = v.y % tM

  for v in [v for v in vehiculos if v.j == "ejey" and v.s=="-"]:
    if v.y < 0:
        v.y = v.y % tM
  for v in vehiculos:
    v.e="libre"
    v.g="No"


def duplicarx(): #to temporarily create a vehicle so the last vehicle in a lane can "see" the first one of the same lane.
  for c in carriles:
    if len([v for v in vehiculos if v.y == c.p and v.j == "ejex" and v.s=="+"])!= 0:
      for v in vehiculos:
        if v.x == min([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p and v.s=="+"]):
          ax = v.vm
          copiax = Vehiculo("doble", min([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p and v.s=="+"])+tM, ax, c.p,"ejex","libre","ejex","+")
          vehiculos.append(copiax)

def duplicarxn():
  for c in carriles:
    if len([v for v in vehiculos if v.y == c.p and v.j == "ejex"and v.s=="-"])!= 0:
      for v in vehiculos:
        if v.x == max([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p and v.s=="-"]):
          axn = v.vm
          copiaxn = Vehiculo("doble", max([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p and v.s=="-"])-tM, axn, c.p,"ejex","libre","ejex","-")
          vehiculos.append(copiaxn)

def duplicary():
  for c in carriles:
    if len([v for v in vehiculos if v.x == c.p and v.j == "ejey" and v.s=="+"])!= 0:
      for v in vehiculos:
        if v.y == min([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p and v.s=="+"]):
          ay = v.vm
          copiay = Vehiculo("doble", c.p, ay, min([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p and v.s=="+"])+tM, "ejey","libre","ejey","+")
          vehiculos.append(copiay)

def duplicaryn():
  for c in carriles:
    if len([v for v in vehiculos if v.x == c.p and v.j == "ejey"and v.s=="-"])!= 0:
      for v in vehiculos:
        if v.y == max([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p and v.s=="-"]):
          ayn = v.vm
          copiayn = Vehiculo("doble", c.p, ayn, max([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p and v.s=="-"])-tM, "ejey","libre","ejey","-")
          vehiculos.append(copiayn)


def lineax(t): #to move vehicles
  global refx
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.j == "ejex" and v.y==c.p and v.s=="+"],[z for z in vehiculos if z.j == "ejex" and z.y==c.p and z.s=="+"][1:]):
      d = z.x - v.x #distance between vehicles
      if v.e == "libre":
        v.f = parametros(v.m, (min(1,max(v.vm + 0.011,0.1))))
        if len([w for w in vehiculos if 0.5<w.x-v.x<1 and abs(w.y-v.y)<0.3 and w.j=="ejey"])!=0: #to stop if there is a vehicle crossing
          v.vm=0
          v.mover(0)
        elif v.fr=="Si" and 10<t-refx and random.random()<0.5: #to stop in green light
          v.vm = 0
          v.mover(0)
          refx=t
        elif d >= v.f*pt:
          v.mover(0.01+0.001*random.uniform(-1,1))
        else:
          v.vm = z.vm
          v.mover(-0.02)
      elif v.e=="parar":
        v.vm = 0
        v.mover(0)
      v.x = v.d + v.x

def lineaxn(t):
  global refxn
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.j == "ejex" and v.y==c.p and v.s=="-"],[z for z in vehiculos if z.j == "ejex" and z.y==c.p and z.s=="-"][1:]):
      d = z.x - v.x
      if z.e == "libre":
        z.f = parametros(z.m, (min(1,max(z.vm + 0.011,0.1))))
        if len([w for w in vehiculos if 0.5<z.x-w.x<1  and abs(w.y-z.y)<0.3 and w.j=="ejey"])!=0:
          z.vm=0
          z.mover(0)
        elif z.fr=="Si" and 10<t-refxn and random.random()<0.5:
          z.vm = 0
          z.mover(0)
          refxn=t
        elif d >= z.f*pt:
          z.mover(0.01+0.001*random.uniform(-1,1))
        else:
          z.vm = v.vm
          z.mover(-0.02)
      elif z.e=="parar":
        z.vm = 0
        z.mover(0)
      z.x = -z.d + z.x

def lineay(t):
  global refy
  for c in carriles:
    vehiculos.sort(key = lambda v: v.y)
    for v,z in zip([v for v in vehiculos if v.j == "ejey" and v.x==c.p and v.s=="+"],[z for z in vehiculos if z.j == "ejey" and z.x==c.p and z.s=="+"][1:]):
      d = z.y - v.y
      if v.e == "libre":
        v.f = parametros(v.m, (min(1,max(v.vm + 0.011,0.1))))
        if len([w for w in vehiculos if abs(w.x-v.x)<0.15 and 0<w.y-v.y<0.6 and z.j=="ejex"])!=0:
          v.vm=0
          v.mover(0)
        elif v.fr=="Si" and 10<t-refy and random.random()<0.5:
          v.vm = 0
          v.mover(0)
          refy=t
        elif d >= v.f*pt:
          v.mover(0.01+0.001*random.uniform(-1,1))
        else:
          v.vm = z.vm
          v.mover(-0.02)
      elif v.e=="parar":
        v.vm = 0
        v.mover(0)
      v.y = v.d + v.y

def lineayn(t):
  global refyn
  for c in carriles:
    vehiculos.sort(key = lambda v: v.y)
    for v,z in zip([v for v in vehiculos if v.j == "ejey" and v.x==c.p and v.s=="-"],[z for z in vehiculos if z.j == "ejey" and z.x==c.p and z.s=="-"][1:]):
      d = z.y - v.y
      if z.e == "libre":
        z.f = parametros(z.m, (min(1,max(z.vm + 0.011,0.1))))
        if len([w for w in vehiculos if abs(w.x-z.x)<0.15 and 0<z.y-w.y<0.6 and w.j=="ejex"])!=0:
          z.vm = 0
          z.mover(0)
        elif z.fr=="Si" and 10<t-refyn and random.random()<0.5:
          z.vm = 0
          z.mover(0)
          refyn=t
        elif d >= z.f*pt:
          z.mover(0.01+0.001*random.uniform(-1,1))
        else:
          z.vm = v.vm
          z.mover(-0.02)
      elif z.e=="parar":
        z.vm = 0
        z.mover(0)
      z.y = -z.d + z.y


def borrar(): #to eliminate the temporarily vehicle
  for c in carriles:
    for v in vehiculos:
      if v.t == "doble":
          vehiculos.remove(v)


def semaforo(t): #traffic lights
  global ti
  for c in carriles:
    for s,s2 in zip([s for s in semafs if s.j == "ejex" and s.t=="carro"],[s2 for s2 in semafs if s2.j == "ejey" and s2.t=="carro"]):
      if s.s == "off" and t == ti+(150*2) or t == 1: #150 is the duration
        s.s="on"
        s2.s ="off"
        ti=t
      elif t == ti+150 and s.s == "on":
        s.s="off"
        s2.s="on"

      for b in [b for b in vehiculos if b.j== "ejex"]:
        for a in [a for a in vehiculos if a.j== "ejey"]:
          if s.s=="on":
            if a.im=="Si" and 19<=a.y<20 and a.s== "+":
              a.fr="Si"
            elif a.im=="Si" and 20.5<a.y<=21.5 and a.s== "-":
              a.fr="Si"
            a.e = "libre"

            if 29<=b.x<30 and b.s== "+":
              if b.im=="No":
                b.e = "parar"
                b.vm = 0
              else:
                pass

            if 30.5<b.x<=31.5 and b.s== "-":
              if b.im=="No":
                b.e = "parar"
                b.vm = 0
              else:
                pass

          elif s2.s=="on":
            if b.im=="Si" and 29<=b.x<30 and b.s== "+":
              b.fr="Si"
            elif b.im=="Si" and 30.5<b.x<=31.5 and b.s== "-":
              b.fr="Si"
            b.e = "libre"
            if 19<=a.y<20 and a.s== "+" :
              if a.im=="No":
                a.e = "parar"
                a.vm = 0
              else:
                pass

            if 20.5<a.y<=21.5 and a.s== "-" :
              if a.im=="No":
                a.e = "parar"
                a.vm = 0
              else:
                pass


def giro(): #to turn in the corner
  for c in carriles:
    for v in vehiculos:
      if v.j != c.j and v.s==c.s:
        if abs(v.x - c.p) <= 0.1 and v.g=="No" and random.random() < 0.5 and v.vm!=0:
          v.j=c.j
          v.x=c.p
          v.g="Si"
        elif abs(v.y - c.p) <= 0.1 and v.g=="No" and random.random() < 0.5 and v.vm!=0:
          v.j=c.j
          v.y=c.p
          v.g="Si"
      else:
        pass


def interaccion(n,s): #all functions are called here
    global filenames
    for i in range(1,t+1):
      semaforo(i)
      giro()
      duplicarx()
      duplicarxn()
      duplicary()
      duplicaryn()
      lineax(i)
      lineaxn(i)
      lineay(i)
      lineayn(i)
      borrar()
      toroide()

      #Collected data from each step to generate graphics, we can change what we collect considering the experiment
      vel1.append([n,st.mean([v.vm for v in vehiculos]),s])
      med1.append([n,st.median([v.vm for v in vehiculos]),s])
      des1.append([n,st.stdev([v.vm for v in vehiculos]),s])
      err1.append([n,ss.sem([v.vm for v in vehiculos]),s])


      #Uncomment this part to create an image of each step and save them to be able to create a GIF:

      '''
      ig, ax = plt.subplots(figsize=(17,17))
      #carros
      plot([v.x for v in vehiculos if v.jo == "ejex" and v.s=="+"], [v.y for v in vehiculos if v.jo == "ejex" and v.s=="+"] ,'o', color="darkorange", markersize=3)
      plot([v.x for v in vehiculos if v.jo == "ejex" and v.s=="-"], [v.y for v in vehiculos if v.jo == "ejex" and v.s=="-"] ,'o', color="deeppink", markersize=3)
      plot([v.x for v in vehiculos if v.jo == "ejey" and v.s=="+"], [v.y for v in vehiculos if v.jo == "ejey" and v.s=="+"] ,'o', color="darkgreen", markersize=3)
      plot([v.x for v in vehiculos if v.jo == "ejey" and v.s=="-"], [v.y for v in vehiculos if v.jo == "ejey" and v.s=="-"] ,'o', color="royalblue", markersize=3)
      plot([v.x for v in vehiculos if v.im=="Si" and v.jo == "ejex"], [v.y for v in vehiculos if v.im=="Si" and v.jo == "ejex"] ,'s', color="cyan", markersize=3)
      plot([v.x for v in vehiculos if v.im=="Si" and v.jo == "ejey"], [v.y for v in vehiculos if v.im=="Si" and v.jo == "ejey"] ,'s', color="cyan", markersize=3)
      plot([v.x for v in vehiculos if v.im=="Si" and v.fr=="Si" and v.jo == "ejex"], [v.y for v in vehiculos if v.im=="Si" and v.fr=="Si" and v.jo == "ejex"] ,'s', color="yellow", markersize=3)
      plot([v.x for v in vehiculos if v.im=="Si" and v.fr=="Si" and v.jo == "ejey"], [v.y for v in vehiculos if v.im=="Si" and v.fr=="Si" and v.jo == "ejey"] ,'s', color="yellow", markersize=3)

      #semaforos de carros
      plot([s.x for s in semafs if s.s== "off" and s.t=="carro"], [s.y for s in semafs if s.s== "off" and s.t=="carro"] ,'d', color="green", markersize=5)
      plot([s.x for s in semafs if s.s== "on" and s.t=="carro"], [s.y for s in semafs if s.s== "on" and s.t=="carro"] ,'d', color="red", markersize=5)
      #semaforos peatonales
      #plot([s.x for s in semafs if s.s== "off" and s.t=="peatonal"], [s.y for s in semafs if s.s== "off" and s.t=="peatonal"] ,'gs', markersize=5)
      #plot([s.x for s in semafs if s.s== "on" and s.t=="peatonal"], [s.y for s in semafs if s.s== "on" and s.t=="peatonal"] ,'rs', markersize=5)
      #plot([s.x for s in semafs if s.p== "Si"], [s.y for s in semafs if s.p== "Si"] ,'ks', markersize=5)
      #carriles
      ax.add_patch(patches.Rectangle((0, 20.25),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((0, 19.75),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((29.75, 0),0.5,50,edgecolor = 'white',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((30.25, 0),0.5,50,edgecolor = 'white',facecolor = 'gray',fill=True))
      text(5,35,"Paso:")
      text(10,35,i)
      text(5,30,(n, "carros"))
      #text(5,28,("Periodo: ",pe))
      tick_params(labelcolor='w')
      #grid(b=True, axis='both', color = "white")
      axis([0,tM,0,tM])

      filename = f'paso{i}.png'
      filenames.append(filename)
      plt.savefig(filename)
      plt.close()
      #'''
      for v in vehiculos:
        v.fr="No"


In [ ]:
#lists
vel1 = []
med1 = []
des1 = []
err1 = []


for i in N:
  for j in range (1,s+1): #the S is the number of simulations
    iniciar(i)
    interaccion(i,j)
  mec=[]
  mec.append(st.mean([l[1] for l in vel1 if l[0]==i]))
  print([mec,i])
  print("-----------------------------","Acabó N =",i,"-----------------------------")


[[0.3042041340219951], 20]
----------------------------- Acabó N = 20 -----------------------------
[[0.22118353237138552], 40]
----------------------------- Acabó N = 40 -----------------------------
[[0.16902328200542116], 60]
----------------------------- Acabó N = 60 -----------------------------


In [ ]:
#If the images were created, we can generate the GIF
gif_name= "name"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#This eliminates all the created images
for filename in set(filenames):
    os.remove(filename)

<ipython-input-78-8b316495292b>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  i = imageio.imread(filename)
